In order to speed up processing time when running classification algorithms, it is often useful to choose only the most "best" genes to use.  There are various algorithms available to choose genes, however here we use Chi2 Select K best.  K is how many genes you wish to use for testing stability.  More genese is usually better, however again in order to speed up processing time we limit the number of genes used.  This program allows you to set a min and max number of genes and an interval.  This will in turn setup numpy arrays with class and the select number of genes for further processing by FASTR and FASTrand.

### Libraries
Must be pre-installed.  Recommended to use virtual environment.

In [21]:
import numpy as np
from enum import Enum
from random import shuffle
from os import path, getcwd
from sklearn.feature_selection import SelectKBest, chi2
from collections import Counter
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import lsqr
from math import sqrt

## Methods and Classes

In [22]:
class SelectionStrategy(Enum):
    KBEST = 1
    RAND = 2

In [23]:
class AlterStrategy(Enum):
    GREEDY = 0
    CHI2 = 1
    RAND = 2
    PERCENT = 3

In [24]:
class Model:

    def __init__(self, samples, eps, class_label):

        self.class_label = class_label
        self.samples = np.array(samples)
        self.eps = eps

        self.correlation = np.corrcoef( self.samples, y=None, rowvar=0, bias=0, ddof=None ) # columns are variables, rows are samples

        self.mask = (np.absolute(self.correlation) > self.eps) # note that the mask is actually the graph

        self.geneFuncMasks = [] #these are the coefficients in Ax=b
        for gene in range(len(self.correlation)):
            currMask = self.mask[gene]
            setOfNeighbors = []
            solutions = []
            for sample in self.samples:
                neighbors = [sample[neighbor] if ((currMask[neighbor] == True) and (gene != neighbor)) else 0 for neighbor in range(len(currMask))]
                neighbors.append(1)
                setOfNeighbors.append(neighbors)
                solutions.append(sample[gene])
            coeff = self.AxbSolver(setOfNeighbors, solutions, 2)
            self.geneFuncMasks.append(coeff.tolist())

        self.coefficients = np.array(self.geneFuncMasks)

    def AxbSolver(self, neighbors, sols, choice):
        # Use lsqr to solve Ax=b
        A=np.array(neighbors)
        b=np.array(sols)
        x = lsqr(A,b)[0]
        return x

    def getExpression(self, sample):
        expression = []
        for gene in range(len(self.coefficients)):
            geneVal = 0
            for neighbor in range(len(self.mask)-1):
                geneVal += self.coefficients[gene][neighbor] * sample[neighbor]
            geneVal += self.coefficients[gene][len(self.mask)]
            expression.append(geneVal)
        return np.array(expression)

    def getClass (self):
        return self.class_label

class NetworkBasedClassifier:

    def __init__(self, epsilon):

        self.models = []
        self.epsilon = epsilon

    def fit ( self, X, y ):
        y = np.array(y)
        X = np.array(X)
        for key in Counter( y ):
            a_class = np.where( y == key )
            self.models.append( Model ( [ X[i] for i in a_class[0] ], self.epsilon, key  ) )

    def predict ( self, X ):
        classifications = []
        for sample in X:
            RMSEs = []
            for model in self.models:
                rmse = sqrt( mean_squared_error( sample, model.getExpression( sample ) ) )
                RMSEs.append(rmse)
            min_index = RMSEs.index( min(RMSEs) )
            label = self.models[min_index].getClass()
            classifications.append(label)
        return np.array(classifications)


###### Enter the series and feature_size to use
Must be all upper case. e.g. `"GSE27562"`

In [20]:
series = "GSE27562"
feature_size = 100
fs_strategy = SelectionStrategy.KBEST

## START MAIN PROGRAM

### Get/Create Directories
Assumes this notebook is in `GenClass-Stability/main/notebooks/`

In [5]:
notebook_dir = getcwd();
main_dir = path.dirname(path.dirname(notebook_dir))
series_dir = path.join(main_dir,"GSE",series)
classes_path = path.join(series_dir, "classes.txt")
exprs_path = path.join(series_dir, "exprs.txt")

### Import Classes and Expressions
Assumes SIT and custome GSE script have been run to import data.

In [6]:
classes =np.loadtxt(classes_path, dtype=np.str, delimiter="\t")
exprs = np.loadtxt(exprs_path, delimiter="\t", unpack=True)

(10094, 125)

In [117]:
def chi2_fs ( genes, classes, num ):
    b = SelectKBest( chi2, num ).fit( genes, classes )
    a = b.get_support( indices = True )
    return a

In [ ]:
def rand_fs( genes, num ):
    b = np.arange(0, num+1, 1)
    random.shuffle(b)

In [118]:
fs_sizes = np.arange(50,101,25)

In [125]:
for fs in fs_sizes:
    indices = chi2_fs(exprs, classes, fs)
    print(indices)

[ 129  273  344  828 1172 1519 1782 1961 2077 2080 2084 2120 2301 2405
 2575 2760 2852 3091 3484 3681 3892 3895 3943 3973 4195 4277 4283 4618
 4677 4954 5218 5229 5370 5768 6140 6213 6495 6611 6930 7141 7187 7907
 8494 8501 8581 9262 9326 9367 9661 9678]
[ 129  273  299  344  828  900  934 1161 1172 1372 1474 1519 1725 1782
 1961 1992 2077 2080 2081 2084 2120 2251 2279 2301 2405 2575 2760 2852
 2866 3091 3311 3484 3681 3881 3892 3895 3943 3973 4049 4069 4195 4277
 4283 4546 4618 4677 4954 5218 5229 5370 5390 5408 5768 6140 6213 6495
 6611 6826 6930 7141 7155 7187 7907 8494 8501 8537 8581 9262 9326 9353
 9367 9557 9661 9678 9845]
[ 129  157  273  299  344  570  761  828  833  900  934  947 1161 1172
 1372 1474 1519 1725 1782 1961 1992 2077 2080 2081 2084 2120 2251 2279
 2301 2405 2575 2760 2852 2866 2876 3091 3125 3137 3141 3184 3311 3393
 3484 3681 3757 3812 3881 3892 3895 3943 3973 4049 4069 4195 4277 4283
 4546 4576 4618 4624 4677 4954 5218 5229 5370 5390 5408 5429 5664 5768
 5933 61

In [127]:
selected = exprs[:, indices]

In [128]:
selected.shape

(125, 100)

In [129]:
selected

array([[4.31152269, 4.4071346 , 5.77272197, ..., 5.83642061, 6.17221809,
        6.24050669],
       [4.7492688 , 3.5923242 , 7.04037055, ..., 5.82981242, 7.350479  ,
        7.94567565],
       [4.43324923, 4.26058516, 5.25456661, ..., 6.98871558, 6.12976211,
        5.41058594],
       ...,
       [4.34395247, 5.26436647, 8.34279915, ..., 4.94476902, 6.67167526,
        6.42701463],
       [4.16553918, 6.06950417, 6.54962712, ..., 4.71671069, 6.30876414,
        6.03027806],
       [3.80756212, 4.93443121, 7.427341  , ..., 4.82043323, 5.45108573,
        5.79667554]])

In [137]:
    indices = chi2_fs(selected, classes, 20)
    print(indices)

[ 4 19 22 32 35 47 48 53 61 62 63 69 71 73 77 78 89 90 93 97]


In [138]:
indices = chi2_fs(exprs,classes,20)
print(indices)

[ 344 1961 2080 2852 3091 3892 3895 4195 4954 5218 5229 5768 6140 6495
 6930 7141 8581 9262 9367 9678]


In [134]:
exprs.shape

(125, 10094)

In [ ]:
exp = np.arange(0, main_size, 1)
random.shuffle(exp)